# Mongo DB + API

## MongoDB

https://motor.readthedocs.io/en/stable/tutorial-asyncio.html

> **Dopo video**
>
> [MongoDB with Python Crash Course - Tutorial for Beginners 2h](https://www.youtube.com/watch?v=E-1xI85Zog8)

In [1]:
!pip install motor

In [2]:
!pip install pymongo

In [3]:
import pandas as pd

def displayData(data):
    df = pd.DataFrame(data)
    display(df)

### Connection

In [4]:
host = '192.168.1.6'
port = '27017'

In [5]:
user = 'root'

In [6]:
import getpass
mongoPassword = getpass.getpass()

 ·········


In [7]:
connectionString = f'mongodb://{user}:{mongoPassword}@{host}:{port}'

### Connection Async

In [9]:
#should be used inside of Tornado library
#import motor.motor_asyncio
#client = motor.motor_tornado.MotorClient(connectionString)

from motor.motor_asyncio import AsyncIOMotorClient
client = AsyncIOMotorClient(connectionString)

print(client)
dbs = list(await client.list_database_names())
print(dbs)

AsyncIOMotorClient(MongoClient(host=['192.168.1.6:27017'], document_class=dict, tz_aware=False, connect=False, driver=DriverInfo(name='Motor', version='2.4.0', platform='asyncio')))
['admin', 'calendar', 'config', 'internetArticles', 'local', 'test_database']


In [10]:
db = client['test_database']

In [11]:
#result = await db.create_collection('test_collection')
#print(result)

### Random Documents

In [12]:
import random
import string

def randomString(prefix='', N=8):
    return prefix + ''.join(random.choices(string.ascii_uppercase, k=N))

defaultStrs = {'name': 'user_', 'group': 'group_'}
defaultNums = {'phone': (602000000, 777999999)}
def randomDocument(strs=defaultStrs, nums=defaultNums):
    result = {}
    for key, value in strs.items():
        result[key] = randomString(value)
    for key, value in nums.items():
        result[key] = random.randint(value[0], value[1])
    return result

def heterogenizeDocument(doc, **values):
    result = {**doc}
    for key, value in values.items():
        if random.random() < 0.5:
            result[key] = randomString(key+'_')
    return result
    
extraFields = {'street': '', 'city': '', 'primarySchool': '', 'secondarySchool': ''}
nums = {**defaultNums, 'age': (15, 80), 'incomeY': (300000, 1500000), 'actualDebt': (0, 10000000)}

def getFullRndDoc():
    return heterogenizeDocument(randomDocument(defaultStrs, nums), **extraFields)

print(getFullRndDoc())
print(getFullRndDoc())

{'name': 'user_OHETGOIN', 'group': 'group_VPXMXGYA', 'phone': 720685438, 'age': 33, 'incomeY': 1116029, 'actualDebt': 7493617, 'street': 'street_STPUIPGC', 'city': 'city_ETKIXLVS', 'primarySchool': 'primarySchool_TSXUAUXC', 'secondarySchool': 'secondarySchool_ZBHHOTJW'}
{'name': 'user_VOPQSPOO', 'group': 'group_CZYUOOSO', 'phone': 679616222, 'age': 43, 'incomeY': 797830, 'actualDebt': 6924139, 'street': 'street_DKGZTOGJ', 'secondarySchool': 'secondarySchool_OEAVFMPD'}


### Create

In [13]:
async def mongoCreateDoc(collection, doc):
    id = await collection.insert_one(doc)
    return await collection.find_one({'_id': id})

### Update

In [14]:
async def mongoUpdateDoc(collection, doc):
    replacementDoc = {**doc}
    del replacementDoc['_id']
    result = await coll.replace_one({'_id': doc['_id']}, replacementDoc)
    return result

### Read

In [15]:
async def mongoReadDoc(collection, doc):
    return await collection.find_one({'_id': doc['_id']})

### Read Multi

In [16]:
async def mongoReadDocs(collection, query={}, skip=0, limit=100):
    cursor = collection.find(query).skip(skip).limit(limit)
    #cursor.sort('i', -1).skip(1).limit(2)
    documents = await cursor.to_list(limit)
    return documents

### Delete

In [17]:
async def mongoDeleteDoc(collection, doc):
    pass

In [18]:
from bson.objectid import ObjectId
collection = client['test_database']['test_collection']

documents = await mongoReadDocs(collection, query={'name': 'user_QHXQNTPD'})
print(documents)

documents = await mongoReadDoc(collection, doc={'_id': ObjectId('606ec05c6b84c76e2a73545f')})
print(documents)

documents = await mongoReadDocs(collection)
displayData(documents)

documents = await mongoReadDocs(collection, skip=25, limit=2)
displayData(documents)

[{'_id': ObjectId('6080497757e0faf209cefc60'), 'name': 'user_QHXQNTPD', 'group': 'group_HWAXFSQG', 'phone': 649929595, 'age': 60, 'incomeY': 986565, 'actualDebt': 73058}]
{'_id': ObjectId('606ec05c6b84c76e2a73545f'), 'key': 'value'}


,_id,key,i,name,group,phone,age,incomeY,actualDebt,street,city,secondarySchool,primarySchool
0,606ec05c6b84c76e2a73545f,value,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,606ec09a6b84c76e2a735460,value,128.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,606ec0a26b84c76e2a735461,value,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,606ecb166b84c76e2a735462,value,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,606ecb4f6b84c76e2a735463,value,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,606ecb4f6b84c76e2a735464,value,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,606ecb4f6b84c76e2a735465,value,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,606ecb4f6b84c76e2a735466,value,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,606ecb4f6b84c76e2a735467,value,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,606ecb4f6b84c76e2a735468,value,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,_id,name,group,phone,age,incomeY,actualDebt,street,city
0,6080497757e0faf209cefc60,user_QHXQNTPD,group_HWAXFSQG,649929595,60,986565,73058,NaN,NaN
1,6080497757e0faf209cefc61,user_ZGZREBJX,group_RQSQJVFP,755276379,39,937108,4960078,street_JLFGTQLT,city_AWKREGGU


## Mongo Map+Reduce

In [20]:
#emit(this.name.substr(0, 4), this.actualDebt);
mapFunction = '''function() {
    if (this.name) {
       emit(0, this.actualDebt);
    }
};'''

reduceFunction ='''function(keyCustId, valuesPrices) {
   return Array.sum(valuesPrices);
};'''

reduced = await db.test_collection.inline_map_reduce(mapFunction, reduceFunction)
print(reduced)

[{'_id': 0.0, 'value': 92017857.0}]


## Fast API

> **Doporučené video**
>
> [FastAPI Introduction - Build Your First Web App - Python Tutorial 12 minut](https://www.youtube.com/watch?v=0RS9W8MtZe4)
>
> [Let's Build a Fast, Modern Python API with FastAPI 1,5 h](https://www.youtube.com/watch?v=sBVb4IB3O_U)

Fast API má jednu obrovskou výhodu oproti obdobným systémům / frameworkům. Touto výhodou je automatická publikace popisu API ve formě **[Swagger](https://swagger.io/)** dokumentu.
Díky Swagger (nebo OpenAPI) je možné využít [celou řadu nástrojů](https://swagger.io/tools/swagger-codegen/) pro generování klientů tvořeného API.

https://fastapi.tiangolo.com/tutorial/sql-databases/

In [21]:
!pip install pydantic

### Schemas

In [22]:
from typing import List, Optional

from pydantic import BaseModel as BaseSchema

class UserCreateSchema(BaseSchema):
    name: str
        
class UserIdSchema(UserCreateSchema):
    id: int

class UserGetSchema(BaseSchema):
    id: int
    name: str
    class Config:
        orm_mode = True #ensures appropriate translation from SQLAlchemy 
    pass

class UserPutSchema(BaseSchema):
    id: int
    name: str


### Server

In [23]:
!pip install uvicorn
!pip install fastapi
!pip install wait4it

#### Minimal Code

In [24]:
import uvicorn
from fastapi import FastAPI

app = FastAPI()#root_path='/api')

def run():
    return uvicorn.run(app, port=9993, host='0.0.0.0', root_path='')

#### Helper Func for Notebook

In [25]:
# Code in this cell is just for (re)starting the API on a Process, and other compatibility stuff with Jupyter cells.
# Just ignore it!

from multiprocessing import Process
from wait4it import wait_for

_api_process = None

def start_api(runNew=True):
    """Stop the API if running; Start the API; Wait until API (port) is available (reachable)"""
    global _api_process
    if _api_process:
        _api_process.terminate()
        _api_process.join()
    
    if runNew:
        _api_process = Process(target=run, daemon=True)
        _api_process.start()
        wait_for(port=9993)

def delete_route(method: str, path: str):
    """Delete the given route from the API. This must be called on cells that re-define a route"""
    [app.routes.remove(route) for route in app.routes if method in route.methods and route.path == path]
    

In [26]:
def delete_all_routes():
    rr = [*app.routes]
    for item in rr:
        app.routes.remove(item)

#### Database CRUD Endpoint

In [27]:
from fastapi import Depends #https://fastapi.tiangolo.com/tutorial/dependencies/
from bson.objectid import ObjectId
from typing import List, Optional

from motor.motor_asyncio import AsyncIOMotorClient

def toJSON(data):
    result = {**data}
    result['_id'] = str(result['_id'])
    return result

async def connectToMongo():
    #client = motor.motor_tornado.MotorClient(connectionString)
    #client = motor.motor_asyncio.AsyncIOMotorClient()
    client = AsyncIOMotorClient(connectionString)
    return client['test_database']

@app.get("/items")
async def getDocsAll(skip: Optional[int]=0, limit: Optional[int]=100, db=Depends(connectToMongo)):
    result = await mongoReadDocs(db.test_collection, {}, skip, limit)
    return list(map(toJSON, result))

@app.get("/items/{id}")
async def getDoc(id: str, db=Depends(connectToMongo)):
    result = await mongoReadDoc(db.test_collection, {'_id': ObjectId(id)})
    return toJSON(result)

@app.post("/items")
async def createDoc(doc, db=Depends(connectToMongo)):
    result = await mongoCreateDoc(db.test_collection, doc)
    return toJSON(result)

@app.put("/items")
async def updateDoc(doc, db=Depends(connectToMongo)):
    result = await mongoUpdateDoc(db.test_collection, doc)
    return toJSON(result)

start_api()

INFO:     Started server process [2245]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:9993 (Press CTRL+C to quit)


INFO:     172.17.0.1:33888 - "GET /docs HTTP/1.1" 200 OK
INFO:     172.17.0.1:33888 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     172.17.0.1:34578 - "GET /docs HTTP/1.1" 200 OK
INFO:     172.17.0.1:34578 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     172.17.0.1:34780 - "GET /items?skip=0&limit=100 HTTP/1.1" 200 OK
INFO:     172.17.0.1:35046 - "GET /items/6080497757e0faf209cefc65 HTTP/1.1" 200 OK


In [9]:
start_api(False)